In [3]:
import os
import pandas
basedir = '/home/idies/workspace/Temporary/raddick/census_scratch/acs5/'
datadir = basedir+'data/'
vardir = basedir+'variables/'
print('Modules imported')

Modules imported


In [9]:
df = pandas.read_csv(datadir+'data_acs2017_for_cra_analysis_mac.csv', low_memory=False, index_col='Unnamed: 0')
df.index.name = 'GEOID'
df.head(100)

,B01001_001,B08013_001,B02001_003,B02001_004,B02001_008,B03001_003,B15001_001,B17001_001,B19013_001,B19062_001
GEOID,,,,,,,,,,
0,1845,20960,96,0,91,44.0,1431.0,1845.0,67826,38020500
1,2172,19065,1184,0,32,23.0,1664.0,1972.0,41287,27463600
2,3385,34000,896,19,83,272.0,2581.0,3385.0,46806,44567700
3,4267,47300,356,20,118,408.0,3299.0,4267.0,55895,82497000
4,9965,104690,1638,0,205,92.0,7437.0,9759.0,68143,304978600
5,3620,26250,907,0,46,107.0,2699.0,3606.0,44549,63152400
6,3420,23125,499,19,91,137.0,2538.0,3389.0,41250,44244600
7,2913,31865,208,43,26,122.0,2177.0,2913.0,80089,68918900
8,11333,131410,1733,0,63,156.0,8283.0,11321.0,64439,219615100


In [ ]:
s = time.time()
states = state_codes_df.index.values.tolist()
states = [x for x in states if x not in ('AS', 'GU', 'MP', 'PR', 'UM', 'VI')]

if (debug >= 1):
    print('Reading shapefiles...')
for x in state_codes_df['STATE'].loc[states].values:
    thefilename = 'tl_2017_{0:02d}_tract.shp'.format(x)
    if (debug >= 2):
        print('Reading file {0:}...'.format(thefilename))
    if (x == 1):
        shapefiles_gdf = geopandas.read_file(shapefiledir+thefilename)
    else:
        this_gdf = geopandas.read_file(shapefiledir+thefilename)
        shapefiles_gdf = pandas.concat((shapefiles_gdf, this_gdf))
        
shapefiles_gdf = shapefiles_gdf.rename(columns={'GEOID': 'PARTIAL_GEOID'})
shapefiles_gdf = shapefiles_gdf.assign(GEOID = shapefiles_gdf['PARTIAL_GEOID'].apply(lambda x: '14000US'+x))

e = time.time()
print('Read shapefiles for {0:,.0f} census tracts in {1:.1f} seconds!'.format(len(shapefiles_gdf), e-s))

print('backing up...')
shapefiles_gdf_bk = shapefiles_gdf

#shapefiles_gdf.sample(1)
print('Done!')

In [ ]:
print('reading from backups...')
data_df = data_df_bk
shapefiles_gdf = shapefiles_gdf_bk

data_df = data_df.set_index('GEOID')
shapefiles_gdf = shapefiles_gdf.set_index('GEOID')

print('joining datasets...')
data_gdf = shapefiles_gdf.join(data_df)

print('backing up...')
data_gdf_bk = data_gdf

print('Done!')

data_gdf.sample(1).T

#data_df.to_csv(datadir+'data_acs_5yr_{0:}.csv'.category)

In [ ]:
print('retrieving from backup...')
data_gdf = data_gdf_bk

data_gdf = data_gdf.assign(statenum = pandas.to_numeric(data_gdf['STATEFP'], errors='coerce'))
data_gdf = data_gdf.assign(countynum = pandas.to_numeric(data_gdf['COUNTYFP'], errors='coerce'))
data_gdf = data_gdf.assign(tractnum = pandas.to_numeric(data_gdf['TRACTCE'], errors='coerce'))

varlist = ['B01001_001', 'B08013_001', 'B02001_003', 'B02001_004', 'B02001_008', 'B03001_003', 'B15001_001', 'B17001_001', 'B19013_001', 'B19062_001']

columns_to_save = ['statenum', 'countynum', 'tractnum', 'NAMELSAD', 'Geography Name']
for x in varlist:
    columns_to_save.append(x)
columns_to_save.append('geometry')

# save only the variables we want
zgdf = data_gdf[columns_to_save]

## save only for Baltimore City
zgdf = zgdf[(zgdf['statenum'] == 24) & (zgdf['countynum'] == 510)]
zgdf = zgdf.to_crs({'init': 'epsg:26985'})

#varlist = ['B01001_001', 'B08013_001', 'B02001_003', 'B02001_004', 'B02001_008', 'B03001_003', 'B15001_001', 'B17001_001', 'B19013_001', 'B19062_001']
#zgdf = zgdf[zgdf['B08013_001'].notnull()]
#for thisvar in varlist:
#    zgdf = zgdf[zgdf[thisvar].notnull()]

#zgdf.groupby('STATEFP').size()
zgdf.groupby('statenum')['B01001_001'].sum()
#zgdf.sample(2)
#metadata_df


In [ ]:
#data_gdf['STATEFP'] = pandas.to_numeric(data_gdf['STATEFP'], errors='coerce')
#data_gdf.dtypes

#zgdf.plot(column='B01001_001')
#plt.show()
metadata_df = metadata_df.set_index('variable')
metadata_df.loc[varlist]

In [ ]:
os.getcwd()
